In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import syft as sy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import time

W0829 19:53:42.702106 4745409984 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0829 19:53:42.855949 4745409984 deprecation_wrapper.py:119] From /Users/lek/miniconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [5]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = 10
        self.lr = 0.02
        self.seed = 1
        self.log_interval = 1 # Log info at each batch
        self.precision_fractional = 3
        self.verbose = True
        self.cuda = True

args = Arguments()

_ = torch.manual_seed(args.seed)

use_cuda = args.cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [6]:
hook = sy.TorchHook(torch)

alice = sy.VirtualWorker(hook, id='alice')
bob = sy.VirtualWorker(hook, id='bob')
charlie = sy.VirtualWorker(hook, id='charlie')

workers = [alice, bob]
crypto_provider = charlie
print(workers)
print(crypto_provider)


W0829 20:09:42.982542 4745409984 hook.py:102] Torch was already hooked... skipping hooking process


[<VirtualWorker id:alice #objects:0>, <VirtualWorker id:bob #objects:0>]
<VirtualWorker id:charlie #objects:0>


In [7]:
# We don't use the whole dataset for efficiency purpose, but feel free to increase these numbers
n_train_items = 640
n_test_items = 640

def get_private_data_loaders(precision_fractional, workers, crypto_provider):
    
    def one_hot_of(index_tensor):
        """
        Transform to one hot tensor
        
        Example:
            [0, 3, 9]
            =>
            [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]
            
        """
        onehot_tensor = torch.zeros(*index_tensor.shape, 10) # 10 classes for MNIST
        onehot_tensor = onehot_tensor.scatter(1, index_tensor.view(-1, 1), 1)
        return onehot_tensor
        
    def secret_share(tensor):
        """
        Transform to fixed precision and secret share a tensor
        """
        return (
            tensor
            .fix_precision(precision_fractional=precision_fractional)
            .share(*workers, crypto_provider=crypto_provider, requires_grad=True)
        )
    
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST('../data/F_MNIST_data', train=True, download=True, transform=transformation),
        batch_size=args.batch_size
    )
    
    private_train_loader = [
        (secret_share(data), secret_share(one_hot_of(target)))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
    
    test_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST('../data/F_MNIST_data', train=False, download=True, transform=transformation),
        batch_size=args.test_batch_size
    )
    
    private_test_loader = [
        (secret_share(data), secret_share(target.float()))
        for i, (data, target) in enumerate(test_loader)
        if i < n_test_items / args.test_batch_size
    ]
    
    return private_train_loader, private_test_loader
    
    
private_train_loader, private_test_loader = get_private_data_loaders(
    precision_fractional=args.precision_fractional,
    workers=workers,
    crypto_provider=crypto_provider
)

100.0%

Extracting ../data/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz


111.0%

Extracting ../data/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz


100.0%

Extracting ../data/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


159.1%

Extracting ../data/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [14]:
def train(args, model, private_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(private_train_loader): # <-- now it is a private dataset
        start_time = time.time()
        
        optimizer.zero_grad()
        
        output = model(data)
        
        # loss = F.nll_loss(output, target)  <-- not possible here
        batch_size = output.shape[0]
        loss = ((output - target)**2).sum().refresh()/batch_size
        
        loss.backward()
        
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            loss = loss.get().float_precision()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.3f}s'.format(
                epoch, batch_idx * args.batch_size, len(private_train_loader) * args.batch_size,
                100. * batch_idx / len(private_train_loader), loss.item(), time.time() - start_time))

In [15]:
def test(args, model, private_test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in private_test_loader:
            start_time = time.time()
            
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum()

    correct = correct.get().float_precision()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct.item(), len(private_test_loader)* args.test_batch_size,
        100. * correct.item() / (len(private_test_loader) * args.test_batch_size)))

In [16]:
model = Net().to(device)
model = model.fix_precision().share(*workers, crypto_provider=crypto_provider, requires_grad=True)

optimizer = optim.SGD(model.parameters(), lr=args.lr)
optimizer = optimizer.fix_precision()

In [17]:
for epoch in range(1, args.epochs + 1):
    train(args, model, private_train_loader, optimizer, epoch)
    test(args, model, private_test_loader)

Train Epoch: 1 [0/640 (0%)]	Loss: 1.072000	Time: 26.362s
Train Epoch: 1 [64/640 (10%)]	Loss: 1.072000	Time: 26.524s
Train Epoch: 1 [128/640 (20%)]	Loss: 1.011000	Time: 26.963s
Train Epoch: 1 [192/640 (30%)]	Loss: 0.963000	Time: 26.966s
Train Epoch: 1 [256/640 (40%)]	Loss: 0.974000	Time: 26.526s
Train Epoch: 1 [320/640 (50%)]	Loss: 0.943000	Time: 26.630s
Train Epoch: 1 [384/640 (60%)]	Loss: 0.923000	Time: 26.716s
Train Epoch: 1 [448/640 (70%)]	Loss: 0.921000	Time: 26.751s
Train Epoch: 1 [512/640 (80%)]	Loss: 0.924000	Time: 26.632s
Train Epoch: 1 [576/640 (90%)]	Loss: 0.892000	Time: 26.775s

Test set: Accuracy: 95.0/640 (15%)

Train Epoch: 2 [0/640 (0%)]	Loss: 0.884000	Time: 26.682s
Train Epoch: 2 [64/640 (10%)]	Loss: 0.887000	Time: 26.626s
Train Epoch: 2 [128/640 (20%)]	Loss: 0.867000	Time: 26.564s
Train Epoch: 2 [192/640 (30%)]	Loss: 0.865000	Time: 26.583s
Train Epoch: 2 [256/640 (40%)]	Loss: 0.865000	Time: 26.586s
Train Epoch: 2 [320/640 (50%)]	Loss: 0.853000	Time: 27.541s
Train Epoch